# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [7]:
## bloque importacion de librerias

import json
import ipywidgets as widgets

import cohere
from IPython.display import display, clear_output

In [8]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


cIsPc5Yag3hdbSXuMR2XyckDtyvgHpP81FI4Rylm


In [10]:
## bloque conexion a Cohere
co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)
response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)
print(response)

id='faa7d1b7-8d42-4bfe-9da0-e4a86d8ccdcb' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [336]:
text_to_analize = """La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


In [25]:
jsonFormat = {
   "paciente": {
       "nombre": "",
       "edad": None
   },
   "fecha_admision": "",
   "sintomas": ["", ""],
   "diagnostico": "",
   "tratamiento": ["", ""]
}

In [42]:
def extract_entities(text):
    response = co.chat(
        model="command",
        messages=[
            {"role": "system", "content":"Eres un asistente especializado en extraer información médica. Devuelve SOLO el JSON sin formato markdown ni backticks."},
            {"role": "user", "content": f"Extrae estas entidades {jsonFormat} del texto: {text}"}
        ]
    )
    json_text = response.message.content[0].text.replace('```json\n', '').replace('\n```', '')
    return json_text


In [43]:
try:
    result = extract_entities(text_to_analize)
    final_result = json.loads(result)
    print(final_result)
except Exception as e:
    print(f"Error: {e}")
    print(f"Respuesta recibida: {result}")

{'paciente': {'nombre': 'María González', 'edad': 45}, 'fecha_admision': '5 de agosto de 2023', 'sintomas': ['fatiga crónica', 'dolores musculares'], 'diagnostico': 'fibromialgia', 'tratamiento': ['fisioterapia', 'medicamentos analgésicos']}


In [323]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [324]:
# your code here
def extract_entities(text):
    response = co.chat(
        model="command",
        messages=[
            {"role": "system", "content":"Eres un asistente especializado en extraer información medica. Devuelve SOLO el JSON sin formato markdown ni backticks. Utiliza solo caracteres ASCII"},
            {"role": "user", "content": f"Extrae estas entidades {jsonFormat}, pueden ser mas o menos segun la informacion del texto: {text}"}
        ]
    )
    json_text = response.message.content[0].text
    #print("Respuesta recibida:", json_text)
    return json_text



In [337]:
# test
try:
    llm_response = extract_entities(text_to_analize)
    final_result = json.loads(llm_response)
    print(final_result)
except Exception as e:
    print(f"Error: {e}")
    print(f"Respuesta recibida: {result}")

{'paciente': {'nombre': 'María González', 'edad': 45, 'fecha_admision': '5 de agosto de 2023', 'sintomas': ['fatiga crónica', 'dolores musculares'], 'diagnostico': 'fibromialgia'}, 'tratamiento': ['fisioterapia', 'medicamentos analgésicos'], 'fecha_consulta': '15 de septiembre'}


## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [158]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."

In [159]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [177]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda

def process(query):
    response = co.chat(
        model="command",
        messages=[{
           "role": "system",
           "content": "Extrae y devuelve SOLO los datos en este formato exacto: nombre|teléfono|email. Sin texto adicional."        
        },{
            "role": "user",
            "content": query
        }]
    )
    try:
        result = response.message.content[0].text
        print("Respuesta:", result)
        name, phone, email = result.split('|')
        return add_contact(name.strip(), phone.strip(), email.strip())
    except ValueError:
        return "Error: formato incorrecto"
# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

In [328]:
test = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
print(process(test))

Contacto añadido con éxito.


In [104]:
####Metodo que acepta un array 
def proccess_for(dictionary_contacts):
    for name, info in dictionary_contacts.items():
        query = f"Agrega a  {name}´con el numero {info['tel']} y el correo {info['mail']}" #Instancio la query aca
        response = co.chat(
        model="command",
        messages=[{
           "role": "system",
           "content": "Extrae nombre, teléfono y email. Responde en formato: nombre|teléfono|email"        
        },{
            "role": "user",
            "content": query
        }]
    )
    name, phone, email = response.message.content[0].text.split('|')  
    print(f"Procesando: {name}") #Para ver la respuesta
    return  add_contact(name, phone, email)

In [103]:
contacts = {
                        'Diego Armando Maradona':{'tel': 15456663258, 'mail': 'DIEGO@gmail.com'},
                      'PELE':{'tel': 1545554178, 'mail': 'PELE@hotmail.com'}
}
print(proccess_for(contacts))
print("Contactos actualizados:", contacts)

Procesando: Pele
Contacto añadido con éxito.
Contactos actualizados: {'Diego Armando Maradona': {'tel': 15456663258, 'mail': 'DIEGO@gmail.com'}, 'PELE': {'tel': 1545554178, 'mail': 'PELE@hotmail.com'}, 'Pele': {'phone': '1545554178', 'email': 'pele@hotmail.com'}}


In [155]:
def proccess_info(query):
    response = co.chat(
        model="command",
        messages=[{
           "role": "system",
           "content": "Devuelve ÚNICAMENTE el nombre y apellido del contacto que se busca, sin explicaciones ni texto adicional."    
        },{
            "role": "user",
            "content": query
        }]
    )
    name = response.message.content[0].text
    print("Nombre del interesado:", name)
    return get_information(name)

In [156]:
test = "Cual es el Email de Joaquin Lopez?"
print(proccess_info(test))

Nombre del interesado: Lopez, Joaquin
Contacto no encontrado.


In [138]:
def identify_action(query):
   response = co.chat(
       model="command",
       messages=[{
           "role": "system",
            "content": "Tu única tarea es identificar si la consulta es para agregar o buscar un contacto. Responde únicamente: 'Agregar' o 'Buscar'"
       },{
           "role": "user",
           "content": query
       }]
   )
   result = response.message.content[0].text
   print("La accion es:", result)
   return result

In [142]:
test = "Agregar a diego armando su numero de telefono = 321546789"
test2 = "Busca la informacion de contacto de diego armando maradona"
test3 = "Cual es el contacto de Diego Armando Maradona?"
print(identify_action(test3))

La accion es: Buscar
Buscar


In [144]:
def process_final(query):
    action =  identify_action(query)

    if action == "Agregar":
        return process(query)
    else:
        return  proccess_info(query)

In [288]:
test = "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
test2 = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
test3 = "Cual es el Email de Juan Pérez.?"
print(process_final(test))

La accion es: Agregar
Contacto añadido con éxito.


In [270]:
def process_request(query):
    response = co.chat(
        model="command",
        messages=[{
            "role": "system", 
            "content": """Eres un asistente que identifica funciones y parámetros de consultas sobre contactos.
            Devuelve un JSON con:
            - action: "add" o "get"
            - name: nombre completo
            - phone: número (para add)
            - email: correo (para add)"""
        },{
            "role": "user",
            "content": query
        }]
    )
    
    data = json.loads(response.message.content[0].text)
    
    if data["action"] == "add":
        result = add_contact(data["name"], data["phone"], data["email"])
        return "Contacto agregado exitosamente"
    else:
        info = get_information(data["name"])
        return f"El email de {data['name']} es {info['email']}" if info != "Contacto no encontrado" else info


In [331]:
test = "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
test2 = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
test3 = "Cual es el Email de Juan Pérez.?"
print(process_request(test3))

El email de Juan Pérez es juanperez@mail.com


## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [291]:
def detect_language(question):
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[{
            "role": "system",
            "content": "Responde SOLO con 'Espanol', 'Ingles' o 'Portugues' según el idioma detectado"
        },{
            "role": "user",
            "content": question
        }]
    )
    return response.message.content[0].text.strip()
    

In [296]:
###Test
test1 = "Cual es tu color favorito?"
test2 = "How are you?"
test3 = "Qual é a sua cor favorita?"

print(detect_language(test2))

Ingles


In [245]:
def verify_relevance(question, story):
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[{
            "role": "system",
            "content": "Determina si la pregunta está relacionada con la historia. Responde solo 'si' o 'no'."
        }, {
            "role": "user",
            "content": f"Historia:\n{story}\n\nPregunta: {question}\n\n¿La pregunta está relacionada con los eventos o personajes de la historia?"
        }]
    )
    return response.message.content[0].text.strip().lower() == 'si'

In [300]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""

 # insertar pregunta relacionada  o  no
pregunta = "¿Cunatos años tiene Thomas?"
pregunta1 = "¿Que le sucedio a thomas?"
pregunta2 = "¿Que autos compiten en la formula 1?"
pregunta3 = "¿Cuantos goles metio Leonel Andres Messi en toda su carrera?"


print(verify_relevance(pregunta3, historia))

False


In [339]:
def history_answer(pregunta, historia):
    # your code here
    language = detect_language(pregunta)
    boolean = verify_relevance(pregunta, historia) 
    if not verify_relevance(pregunta, historia):
        responses = {
            'Espanol': "Lo siento no puedo ayudarte con eso.",
            'Ingles': "Sorry, I can't help you with that.",
            'Portugues': "Desculpe, não posso ajudar com isso."
        }
        return f"{responses[language]} ¡Hakuna Matata!"

    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[{
            "role": "system",
            "content": """Eres un analizador de textos que:
            1. Solo responde preguntas sobre la historia proporcionada
            2. Usa emojis relacionados
            3. Responde en tercera persona
            4. Usa una sola oración
            5. Responde segun el idioma de origen"""
            
        },{
            "role": "user",
            "content": f"Historia: {historia}\nPregunta: {pregunta}\nIdioma: {language}"
        }]
    )

    result = response.message.content[0].text.strip()
    print(pregunta, language)
    return f"{result}, ¡Hakuna Matata!."

In [341]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""

 # insertar pregunta relacionada  o  no
pregunta = "¿Cuantos años tiene Thomas?"
pregunta1 = "¿Que hizo el rey con el pequeño thomas?"
pregunta2 = "¿Que autos compiten en la formula 1?"
pregunta3 = "¿Cuantos goles metio Leonel Andres Messi en toda su carrera?"

question0 = "How old is Thomas?"
question1 = "What did the king do with young Thomas?"
question2 = "What cars compete in Formula 1?"
question3 = "How many goals has Lionel Andres Messi scored in his career?"



# respuesta
print(history_answer(question2, historia))

Sorry, I can't help you with that. ¡Hakuna Matata!


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [3]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

# chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    responses = {
        "hola": "¡Hola! ¿En qué puedo ayudarte?",
        "adiós": "¡Hasta luego!",
    }

    return responses.get(message.lower(), "Lo siento, no entiendo esa pregunta.")

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [319]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    chat_history.append({"role": "user", "content": message})

    response = co.chat(
        model="command",
        messages=[{
            "role": "system", 
            "content": """Eres un tutor entunciasta que:
            1. Solo responde maximo 70 tokens
            2. Da consejos utiles
            3. Responde solo en el idioma de origen de la pregunta
            4. Responde solo en 2 oraciones
            5. Manten el contenido de la respuesta dentro de los 70 Tokens
            6. Limitate a constestar lo que se te pregunta'"""
        }] + chat_history,
        max_tokens=70
    )

    bot_messages = response.message.content[0].text
    chat_history.append({"role": "assistant", "content": bot_messages})
    return bot_messages

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [320]:
print(chat_history)

[{'role': 'user', 'content': 'Haz visto Rick y Morty?'}, {'role': 'assistant', 'content': '¡Hola amigo! Si, ya he visto muchos episodios de Rick y Morty. Me encanta la serie por su sabor cruzado de comedia, aventura y fantasia. \n\nAdemas,  el personaje de Rick Sanchez es una caricatura de un hombre'}, {'role': 'user', 'content': 'Ahora dilo en ingles'}, {'role': 'assistant', 'content': "Sure! I think that's a great idea and I can definitely help you with that. \n\nI am very passionate about Rick and Morty and I would love to help you express that in English. Here are some of the reasons why I think Rick and Morty is such an exciting show and why it has become very popular: \n\n1"}, {'role': 'user', 'content': 'Y en protuges?'}, {'role': 'assistant', 'content': "I'm sorry, I can only respond in English as per my guidelines. If you have any further questions or want to discuss anything else, please ask!"}]


### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.